In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mlsp2-feature/model3_features.h5
/kaggle/input/test-set/test/X/melspec_349.npy
/kaggle/input/test-set/test/X/melspec_2250.npy
/kaggle/input/test-set/test/X/melspec_1843.npy
/kaggle/input/test-set/test/X/melspec_1328.npy
/kaggle/input/test-set/test/X/melspec_1587.npy
/kaggle/input/test-set/test/X/melspec_501.npy
/kaggle/input/test-set/test/X/melspec_1568.npy
/kaggle/input/test-set/test/X/melspec_2180.npy
/kaggle/input/test-set/test/X/melspec_1776.npy
/kaggle/input/test-set/test/X/melspec_1804.npy
/kaggle/input/test-set/test/X/melspec_1962.npy
/kaggle/input/test-set/test/X/melspec_255.npy
/kaggle/input/test-set/test/X/melspec_1623.npy
/kaggle/input/test-set/test/X/melspec_390.npy
/kaggle/input/test-set/test/X/melspec_1375.npy
/kaggle/input/test-set/test/X/melspec_227.npy
/kaggle/input/test-set/test/X/melspec_1138.npy
/kaggle/input/test-set/test/X/melspec_880.npy
/kaggle/input/test-set/test/X/melspec_2201.npy
/kaggle/input/test-set/test/X/melspec_798.npy
/kaggle/input/test-s

In [2]:
import tensorflow as tf
import os
import numpy as np
import librosa
from sklearn.model_selection import train_test_split
lst_names = os.listdir("../input/assign-2-val/X_new")
file_names = []
for m in range(len(lst_names)):
    file_names.append(lst_names[m][9:])

In [ ]:
x_train = []
for m in range(10000):
    z = np.load("../input/assign-2-train/X_new/melspec1_" + str(m) + ".npy")
    z = z.T
    z = np.reshape(z,(1000,64,1))
    features = librosa.feature.mfcc(S=librosa.power_to_db(z), n_mfcc = 32)
    x_train.append(features)
for m in range(2000):
    z = np.load("../input/assign-2-val/X_new/melspec1_" + file_names[m])
    z = z.T
    z = np.reshape(z,(1000,64,1))
    features = librosa.feature.mfcc(S=librosa.power_to_db(z), n_mfcc = 32)
    x_train.append(features)
x_train = np.array(x_train)
x_train.shape 

In [ ]:
y_train = []
for m in range(10000):
    y_train.append(np.load("../input/assign-2-train/Y_new/eventroll1_" + str(m) + ".npy"))
    print(m)
for m in range(2000):
    y_train.append(np.load("../input/assign-2-val/Y_new/eventroll1_" + file_names[m]))
y_train = np.array(y_train)
y_train.shape

In [5]:
from sklearn.utils import shuffle
x_train, y_train = shuffle(x_train, y_train)
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, random_state=42, test_size=0.167)

In [ ]:
(temp, f, t, num) = x_train.shape 
(temp, f, t, num)

In [3]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization

In [13]:
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation="leaky_relu", input_shape=(f, t, num)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(256, activation='leaky_relu'))
model.add(BatchNormalization())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='sigmoid'))

In [14]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 998, 30, 32)       320       
_________________________________________________________________
batch_normalization_3 (Batch (None, 998, 30, 32)       128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 499, 15, 32)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 497, 13, 64)       18496     
_________________________________________________________________
batch_normalization_4 (Batch (None, 497, 13, 64)       256       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 248, 6, 64)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 248, 6, 64)       

In [15]:
model.compile(
    loss='binary_crossentropy',
    optimizer=tf.keras.optimizers.Adam(),
    metrics=[tf.keras.metrics.Precision(),tf.keras.metrics.Recall(), ],
)

history = model.fit(x_train, y_train, verbose = 1, epochs=40, batch_size = 32, validation_data=(x_valid, y_valid))

2022-11-12 11:26:20.435750: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1280000000 exceeds 10% of free system memory.
2022-11-12 11:26:21.861500: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1280000000 exceeds 10% of free system memory.
2022-11-12 11:26:22.921815: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/40


2022-11-12 11:26:24.895784: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


313/313 [==============================] - ETA: 0s - loss: 0.4314 - precision: 0.6007 - recall: 0.5013

2022-11-12 11:26:38.031111: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 256000000 exceeds 10% of free system memory.


313/313 [==============================] - 16s 26ms/step - loss: 0.4314 - precision: 0.6007 - recall: 0.5013 - val_loss: 0.3522 - val_precision: 0.7930 - val_recall: 0.4113
Epoch 2/40
313/313 [==============================] - 8s 24ms/step - loss: 0.3295 - precision: 0.7777 - recall: 0.5514 - val_loss: 0.3404 - val_precision: 0.7483 - val_recall: 0.4782
Epoch 3/40
313/313 [==============================] - 7s 24ms/step - loss: 0.2922 - precision: 0.8164 - recall: 0.6044 - val_loss: 0.3600 - val_precision: 0.8031 - val_recall: 0.4470
Epoch 4/40
313/313 [==============================] - 8s 24ms/step - loss: 0.2582 - precision: 0.8357 - recall: 0.6526 - val_loss: 0.3304 - val_precision: 0.7641 - val_recall: 0.5042
Epoch 5/40
313/313 [==============================] - 7s 24ms/step - loss: 0.2341 - precision: 0.8462 - recall: 0.6920 - val_loss: 0.3132 - val_precision: 0.8035 - val_recall: 0.5362
Epoch 6/40
313/313 [==============================] - 8s 24ms/step - loss: 0.2110 - precision: 

In [20]:
model.save('model3_features.h5')

In [16]:
yhat_probs = model.predict(x_valid, verbose=1)
yhat1 = yhat_probs.round()

2022-11-12 11:31:41.745049: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 256000000 exceeds 10% of free system memory.


63/63 [==============================] - 0s 5ms/step


In [4]:
events_types = ['Alarm_bell_ringing', 'Blender','Cat','Dishes','Dog','Electric_shaver_toothbrush','Frying','Running_water','Speech','Vacuum_cleaner']

In [18]:
from sklearn.metrics import classification_report
report = classification_report(y_valid, yhat1, output_dict=True, target_names = events_types)
report_df = pd.DataFrame(report).transpose()

/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [17]:
report_df

,precision,recall,f1-score,support
Alarm_bell_ringing,0.488636,0.388554,0.432886,332.0
Blender,0.683486,0.454268,0.545788,328.0
Cat,0.622951,0.497817,0.553398,229.0
Dishes,0.557895,0.321212,0.407692,495.0
Dog,0.760736,0.388715,0.514523,319.0
Electric_shaver_toothbrush,0.849711,0.746193,0.794595,197.0
Frying,0.921429,0.638614,0.754386,202.0
Running_water,0.823529,0.428571,0.563758,196.0
Speech,0.873342,0.851876,0.862475,1546.0
Vacuum_cleaner,0.910569,0.589474,0.715655,190.0


In [5]:
model1 = tf.keras.models.load_model('../input/mlsp2-feature/model3_features.h5')

2022-11-13 09:56:08.650750: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-13 09:56:08.804598: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-13 09:56:08.805737: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-13 09:56:08.808156: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [9]:
x_test = []
for m in range(2500):
    z = np.load("../input/test-set/test/X/melspec_" + str(m) + ".npy")
    z = z.T
    z = np.reshape(z,(1000,64,1))
    features = librosa.feature.mfcc(S=librosa.power_to_db(z), n_mfcc = 32)
    x_test.append(features)
x_test = np.array(x_test)
x_test.shape

(2500, 1000, 32, 1)

In [6]:
model1.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 998, 30, 32)       320       
_________________________________________________________________
batch_normalization_3 (Batch (None, 998, 30, 32)       128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 499, 15, 32)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 497, 13, 64)       18496     
_________________________________________________________________
batch_normalization_4 (Batch (None, 497, 13, 64)       256       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 248, 6, 64)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 248, 6, 64)       

In [11]:
yhat_probs = model1.predict(x_test, verbose=1)
yhat1 = yhat_probs.round()
print(yhat1)
print(len(yhat1))

2022-11-13 09:58:06.567269: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-11-13 09:58:07.603812: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


79/79 [==============================] - 7s 6ms/step
[[0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 ...
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 1. 0.]]
2500


In [12]:
lst = []
c = 0
for m in yhat1:
    z = []
    if m[0] == 1:
        z.append(events_types[0])
    if m[1] == 1:
        z.append(events_types[1])
    if m[2] == 1:
        z.append(events_types[2])
    if m[3] == 1:
        z.append(events_types[3])
    if m[4] == 1:
        z.append(events_types[4])
    if m[5] == 1:
        z.append(events_types[5])
    if m[6] == 1:
        z.append(events_types[6])
    if m[7] == 1:
        z.append(events_types[7])
    if m[8] == 1:
        z.append(events_types[8])
    if m[9] == 1:
        z.append(events_types[9])
    lst.append(z)

In [13]:
lst

[['Dog', 'Speech'],
 ['Dishes', 'Frying', 'Speech'],
 ['Electric_shaver_toothbrush', 'Speech'],
 ['Alarm_bell_ringing', 'Speech'],
 ['Speech'],
 ['Alarm_bell_ringing', 'Speech'],
 ['Speech'],
 ['Speech', 'Vacuum_cleaner'],
 ['Alarm_bell_ringing', 'Cat', 'Speech'],
 ['Cat', 'Speech'],
 ['Alarm_bell_ringing', 'Speech'],
 ['Speech'],
 ['Speech'],
 ['Frying', 'Speech'],
 ['Speech'],
 ['Speech'],
 ['Speech'],
 ['Cat', 'Speech'],
 ['Blender', 'Speech'],
 ['Speech', 'Vacuum_cleaner'],
 ['Dishes', 'Frying', 'Speech'],
 ['Electric_shaver_toothbrush', 'Speech'],
 ['Alarm_bell_ringing', 'Speech'],
 ['Speech'],
 ['Speech'],
 ['Alarm_bell_ringing', 'Speech'],
 ['Dishes', 'Speech'],
 ['Dog', 'Speech'],
 ['Speech'],
 ['Speech'],
 ['Electric_shaver_toothbrush', 'Speech'],
 ['Dishes', 'Frying', 'Speech'],
 ['Dog', 'Speech'],
 ['Alarm_bell_ringing', 'Speech'],
 ['Alarm_bell_ringing', 'Speech'],
 ['Running_water', 'Speech'],
 ['Speech'],
 ['Dishes', 'Frying', 'Speech'],
 ['Cat', 'Speech'],
 ['Dog', 'Spee

In [15]:
lst_names = []
for m in range(2500):
    lst_names.append("melspec_" + str(m) + ".npy")
lst_names

['melspec_0.npy',
 'melspec_1.npy',
 'melspec_2.npy',
 'melspec_3.npy',
 'melspec_4.npy',
 'melspec_5.npy',
 'melspec_6.npy',
 'melspec_7.npy',
 'melspec_8.npy',
 'melspec_9.npy',
 'melspec_10.npy',
 'melspec_11.npy',
 'melspec_12.npy',
 'melspec_13.npy',
 'melspec_14.npy',
 'melspec_15.npy',
 'melspec_16.npy',
 'melspec_17.npy',
 'melspec_18.npy',
 'melspec_19.npy',
 'melspec_20.npy',
 'melspec_21.npy',
 'melspec_22.npy',
 'melspec_23.npy',
 'melspec_24.npy',
 'melspec_25.npy',
 'melspec_26.npy',
 'melspec_27.npy',
 'melspec_28.npy',
 'melspec_29.npy',
 'melspec_30.npy',
 'melspec_31.npy',
 'melspec_32.npy',
 'melspec_33.npy',
 'melspec_34.npy',
 'melspec_35.npy',
 'melspec_36.npy',
 'melspec_37.npy',
 'melspec_38.npy',
 'melspec_39.npy',
 'melspec_40.npy',
 'melspec_41.npy',
 'melspec_42.npy',
 'melspec_43.npy',
 'melspec_44.npy',
 'melspec_45.npy',
 'melspec_46.npy',
 'melspec_47.npy',
 'melspec_48.npy',
 'melspec_49.npy',
 'melspec_50.npy',
 'melspec_51.npy',
 'melspec_52.npy',
 'm

In [16]:
import pandas as pd
df = pd.DataFrame(list(zip(lst_names, lst)),
               columns =['fileid', 'prediction'])
df

,fileid,prediction
0,melspec_0.npy,"[Dog, Speech]"
1,melspec_1.npy,"[Dishes, Frying, Speech]"
2,melspec_2.npy,"[Electric_shaver_toothbrush, Speech]"
3,melspec_3.npy,"[Alarm_bell_ringing, Speech]"
4,melspec_4.npy,[Speech]
...,...,...
2495,melspec_2495.npy,[Speech]
2496,melspec_2496.npy,[Speech]
2497,melspec_2497.npy,"[Dishes, Frying, Speech]"
2498,melspec_2498.npy,"[Electric_shaver_toothbrush, Speech]"


In [17]:
df.to_csv("model_2_test_result.csv")